In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as FT
import torchvision.datasets as datasets
from torch.utils.data import DataLoader as dataloader
import torchvision.models as models

import time
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import trange, tqdm
from PIL import Image
import itertools
import copy

from Trainer import ModelTrainer

In [ ]:
# The size of our mini batches
batch_size = 64

# How many itterations of our dataset
num_epochs = 30

# Optimizer learning rate
learning_rate = 1e-4

# Where to load/save the dataset from 
# data_set_root = "../../datasets"
data_set_root = "/media/luke/Quick Storage/Data"

# What to resize our images to 
image_size = 96

In [ ]:
start_from_checkpoint = False

save_dir = '../data/Models'
model_name = 'ResNet18_STL10'

In [ ]:
# Set device to GPU_indx if GPU is avaliable
GPU_indx = 0
device = torch.device(GPU_indx if torch.cuda.is_available() else 'cpu')

# Data Augmentation Transform
After training ResNet with no augmentations record the results, then implement the data augmentation. <br>
With a small dataset our large model will more then likely simply overfit to (or memorize) the training data which will often lead to bad evaluation results<br>
We can "create more" data from our limited dataset by applying random transformations as we sample images from our dataset instead of simply resizing them<br>
By applying these transformations we are also forcing our model to generalise better to unseen images<br>
You can also apply random affine transformations (shifts, scaling, rotations etc) - see <a href="https://pytorch.org/vision/0.12/">Pytorch documentations</a>.<br>
NOTE: you should only apply transforms that make sense, eg if at test time you'll never see an upside-down cat, don't flip your images vertically 


In [ ]:
transform = transforms.Compose([transforms.Resize(image_size),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                     std=[0.229, 0.224, 0.225])])

In [ ]:
class RotateSTL10(datasets.STL10):
    all_perms = [-90, -45, 0, 45, -90]
    
    def __getitem__(self, index):
            
            img = self.data[index]
            rand_int = random.randint(0, len(self.all_perms) - 1)

            # doing this so that it is consistent with all other datasets
            # to return a PIL Image
            img = Image.fromarray(np.transpose(img, (1, 2, 0)))
            
            img = FT.rotate(img, angle=self.all_perms[rand_int])

            if self.transform is not None:
                img = self.transform(img)

            return img, rand_int

In [ ]:
class ShuffleSTL10(datasets.STL10):
    puzzel_size = 2
    
    iter_array = itertools.permutations(np.arange(puzzel_size**2))
    all_perms = []
    for arr in iter_array:
        all_perms.append(torch.tensor([arr]))

    def __getitem__(self, index):
            img = self.data[index]
            # doing this so that it is consistent with all other datasets
            # to return a PIL Image
            img = Image.fromarray(np.transpose(img, (1, 2, 0)))
            if self.transform is not None:
                img = self.transform(img)
                
            img_size = img.shape[-1]
            puzzel_sections = self.puzzel_size**2
            
            unshuffle = nn.PixelUnshuffle(img_size//self.puzzel_size)
            shuffle = nn.PixelShuffle(img_size//self.puzzel_size)
            rand_int = random.randint(0, len(self.all_perms) - 1)
            perm = self.all_perms[rand_int]
            
            img_out = unshuffle(img.unsqueeze(0))
            img_out = img_out.reshape(1, img.shape[0], -1, puzzel_sections)
            img_out = shuffle(img_out[:, :, :, perm].reshape(1, -1, 
                                                                  self.puzzel_size, 
                                                                  self.puzzel_size))

            return img_out.squeeze(0), rand_int

# Create the training, testing and validation data 

In [ ]:
# Define our STL10 Datasets
# https://pytorch.org/docs/stable/torchvision/datasets.html#torchvision.datasets.STL10

# Dataset definition is a bit differenet to MNIST and CIFAR10
# STL10 has 3 different datasets, test, train and unlabeled
# http://ai.stanford.edu/~acoates/stl10/
# training set only has 5000 images and test set only 8000
# Image size in this dataset are 96x96, larger then what we've been using

train_data = TripletSTL10(data_set_root, split='train+unlabeled', download=True, transform=transform)
test_data = TripletSTL10(data_set_root, split='test', download=True, transform=transform)

# Split trainging data into train and validation set with 90/10% traning/validation split
validation_split = 0.9

n_train_examples = int(len(train_data)*validation_split)
n_valid_examples = len(train_data) - n_train_examples
train_data, valid_data = torch.utils.data.random_split(train_data, [n_train_examples, n_valid_examples],
                                                       generator=torch.Generator().manual_seed(42))

# Create the Pretrained Network

Use Pytorch's implementation of [ResNet18](https://pytorch.org/vision/0.14/models.html) with both randomly initialised weights and pretrained weights.

In [ ]:
# Create an instance of the ResNet Model
res_net = models.resnet18().to(device)

In [ ]:
class TripletResNet(nn.Module):
    def __init__(self, resnet):
        super(TripletResNet, self).__init__()
        self.resnet = resnet
        
    def forward(self, x):
        return x

In [ ]:
num_outputs = len(train_data.dataset.all_perms)

model_trainer = ModelTrainer(model=res_net, output_size=num_outputs, device=device, 
                             loss_fun=nn.CrossEntropyLoss(), 
                             batch_size=batch_size, learning_rate=learning_rate, 
                             save_dir=save_dir, model_name=model_name, 
                             start_from_checkpoint=start_from_checkpoint)

In [ ]:
model_trainer.set_data(train_set=train_data, test_set=test_data, val_set=valid_data)

# Set a Learning Rate Scheduler
We can dynamically change the <a href="https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate">learning rate</a> during training to help our model converge to a better minimum!

In [ ]:
# Uncomment one of these and try it out!

# model_trainer.set_lr_schedule(optim.lr_scheduler.StepLR(model_trainer.optimizer, 
#                                                         step_size=1, 
#                                                         gamma=0.95))

# model_trainer.set_lr_schedule(optim.lr_scheduler.CosineAnnealingLR(model_trainer.optimizer, 
#                                                                    T_max=num_epochs, 
#                                                                    eta_min=0))

# View Data

In [ ]:
plt.figure(figsize = (20,10))
images, labels = next(iter(model_trainer.train_loader))
out = torchvision.utils.make_grid(images[0:16], normalize=True)
_ = plt.imshow(out.numpy().transpose((1, 2, 0)))

In [ ]:
# Lets see how many Parameter's our Model has!
num_params = 0
for param in model_trainer.model.parameters():
    num_params += param.flatten().shape[0]
print("This model has %d (approximately %d Million) Parameters!" % (num_params, num_params//1e6))

# Train Model!
Our full training method is now fully contained within the trainner class! Simply run the run_training method and specify how many epochs it should train for!

In [ ]:
model_trainer.run_training(num_epochs=num_epochs)

# View Results

In [ ]:
print("The highest validation accuracy was %.2f%%" %(model_trainer.best_valid_acc*100))

In [ ]:
_ = plt.figure(figsize = (10,5))
train_x = np.linspace(0, num_epochs, len(model_trainer.train_loss_logger))
_ = plt.plot(train_x, model_trainer.train_loss_logger)
_ = plt.title("Training Loss")

In [ ]:
_ = plt.figure(figsize = (10,5))
train_x = np.linspace(0, num_epochs, len(model_trainer.train_acc_logger))
_ = plt.plot(train_x, model_trainer.train_acc_logger, c = "y")
valid_x = np.linspace(0, num_epochs, len(model_trainer.val_acc_logger))
_ = plt.plot(valid_x, model_trainer.val_acc_logger, c = "k")

_ = plt.title("Accuracy")
_ = plt.legend(["Training accuracy", "Validation accuracy"])

# Evaluate

In [ ]:
# Call the evaluate function and pass the evaluation/test dataloader etc
test_acc = model_trainer.evaluate_model(train_test_val="test")
print("The Test Accuracy is: %.2f%%" %(test_acc*100))